In [ ]:
# Dictionary to store the member numbers

number_members = pd.read_excel(
    filename,
    sheet_name = "Structure - Members",
    usecols="A",
    header=1,
    skiprows=[2]
)

member_list = []
for index, row in number_members.iterrows():
    member_list.append(row['Memb'])
    
# member_list
    
# Dictionary to store the Node A numbers

number_node_a = pd.read_excel(
    filename,
    sheet_name = "Structure - Members",
    usecols="F",
    header=1,
    skiprows=[2]
)

node_a_list = []
for index, row in number_node_a.iterrows():
    node_a_list.append(row['Node A'])
    
# node_a_list    
    
# Dictionary to store the Node B numbers

number_node_b = pd.read_excel(
    filename,
    sheet_name = "Structure - Members",
    usecols="G",
    header=1,
    skiprows=[2]
)

node_b_list = []
for index, row in number_node_b.iterrows():
    node_b_list.append(row['Node B'])
    
# node_b_list      
 

# Loop to get unique members attached to each node    

# for node, idx in node_list:
#     if node_list[idx] == node_a_list[idx]:
        


# dictionary = {member_no = XX,
#               member_node_1 = XX,
#               axial_force_1 = XX,
#               y_axis_shear_1 = XX,
#               z_axis_shear_1 = XX,
#               x_axis_torsion_1 = XX,
#               y_axis_moment_1 = XX,
#               z_axis_moment_1 = XX,
#               member_node_2 = XX,
#               axial_force_2 = XX,
#               y_axis_shear_2 = XX,
#               z_axis_shear_2 = XX,
#               x_axis_torsion_2 = XX,
#               y_axis_moment_2 = XX,
#               z_axis_moment_2 = XX,
#              }

# for case in case_list:
#     with pd.ExcelWriter("SGLoadsExtract.xlsx") as workbook:
#         df.to_excel(workbook, sheet_name=f'Load Case {case}', index=False)

In [20]:
import pandas as pd

pd.set_option('display.max_rows', None)

filename = "Test Model for TV.xlsx"

# Extracts case number into a list

number_cases = pd.read_excel(
    filename,
    sheet_name = "Loads - Load Case Titles",
    usecols="A",
    header=1,
    skiprows=[2]
)
   
case_list = []
for index, row in number_cases.iterrows():
    case_list.append(row['Case'])

# Extracts case titles into a list

title_cases = pd.read_excel(
    filename,
    sheet_name = "Loads - Load Case Titles",
    usecols="B",
    header=1,
    skiprows=[2]
)    
    
case_title_list = []
for index, row in title_cases.iterrows():
    case_title_list.append(row['Title'])    
    
print(case_title_list)       
    
# Extracts number of nodes into a list    
        
number_nodes = pd.read_excel(
    filename,
    sheet_name = "Structure - Nodes",
    usecols="A",
    header=0,
    skiprows=[1]
)

node_list = []
for index, row in number_nodes.iterrows():
    node_list.append(row['Node'])

# print(node_list)
    
# Dictionary to store DataFrames for each load case    

loads_dict = {}

for load_case in case_list:
    key = f'Case_{load_case}'
    loads_dict[key] = pd.read_excel(filename,
                                    sheet_name = f"... Forces and Moments - Case {load_case}",
                                    header=[1],
                                    skiprows=[2]
                                    )
    
for load_case, df in loads_dict.items():
    # Replace NaN values in the 'Memb' column with forward fill
    df['Memb'] = df['Memb'].ffill()

    # Update the DataFrame in the dictionary
    loads_dict[load_case] = df
    
# loads_dict['Case_2']
    



['Horizontal Load', 'Vertical Load']


In [2]:
number_members_nodes = pd.read_excel(
    filename,
    sheet_name = "Structure - Members",
    usecols="A,F,G",
    header=1,
    skiprows=[2]
)

# Dictionary to store results
node_members_dict = {}

# Loop through nodes
for node in node_list:
    # Filter rows
    filtered_rows = number_members_nodes[(number_members_nodes['Node A'] == node) | (number_members_nodes['Node B'] == node)]
    
    # Extract unique members
    unique_members = filtered_rows['Memb'].unique().tolist()
    
    # Add to dictionary
    node_members_dict[f'Node_{node}'] = unique_members

# Print result
# for key, value in node_members_dict.items():
#     print(f"{key}: {value}")

print(node_members_dict)

{'Node_1': [1, 3], 'Node_2': [1, 2, 4], 'Node_3': [3, 4, 5, 7], 'Node_4': [2, 5, 6, 8], 'Node_5': [7, 8, 9, 11, 12], 'Node_6': [6, 9, 10], 'Node_7': [11, 13, 15, 16], 'Node_8': [10, 12, 13, 14], 'Node_9': [15, 17], 'Node_10': [14, 16, 17]}


In [17]:
# Create a new DataFrame to store values
result_df = pd.DataFrame()

# Iterate through rows of the original DataFrame
for load_case, df in loads_dict.items():
    for index, row in df.iterrows():
        node = int(row['Node'])
        memb = int(row['Memb'])
    
        # Check if the combination of 'Node' and 'Memb' is present in the dictionary
        key = f'Node_{node}'
        if key in node_members_dict and memb in node_members_dict[key]:
            
            # Extract specific values and add them to the result DataFrame
            values = row[['Node','Memb','Force', 'Shear', 'Shear.1', 'Torsion', 'Moment', 'Moment.1']].tolist()
            values.append(load_case)
            result_df = pd.concat([result_df, pd.DataFrame([values], columns=['Node Connection', 'Memb', 'Axial Force', 'Y-Axis Shear', 'Z-Axis Shear', 'X-Axis Torsion', 'Y-Axis Moment', 'Z-Axis Moment', 'Load Case'])], ignore_index=True)

# Drop the index column
result_df = result_df.reset_index(drop=True)
            
# Sort dataframe by nodes
result_df = result_df.sort_values(by=['Node Connection', 'Load Case', 'Memb'])

# Display the result DataFrame
result_df

,Node Connection,Memb,Axial Force,Y-Axis Shear,Z-Axis Shear,X-Axis Torsion,Y-Axis Moment,Z-Axis Moment,Load Case
0,1.0,1.0,-2.571,0.000,0.0,0.0,0.0,0.000,Case_1
4,1.0,3.0,-10.000,0.071,0.0,0.0,0.0,0.000,Case_1
34,1.0,1.0,5.004,0.000,0.0,0.0,0.0,0.000,Case_2
38,1.0,3.0,0.000,-0.004,0.0,0.0,0.0,0.000,Case_2
1,2.0,1.0,-2.571,0.000,0.0,0.0,0.0,0.000,Case_1
2,2.0,2.0,7.372,0.057,0.0,0.0,0.0,0.000,Case_1
6,2.0,4.0,3.717,0.000,0.0,0.0,0.0,0.000,Case_1
35,2.0,1.0,5.004,0.000,0.0,0.0,0.0,0.000,Case_2
36,2.0,2.0,5.052,-0.048,0.0,0.0,0.0,0.000,Case_2
40,2.0,4.0,-7.144,0.000,0.0,0.0,0.0,0.000,Case_2
